In [13]:
# Basic Import
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt 
import seaborn as sns
# Modelling
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.neighbors import KNeighborsRegressor
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor,AdaBoostRegressor
from sklearn.svm import SVR
from sklearn.linear_model import LinearRegression, Ridge,Lasso
from sklearn.metrics import r2_score, mean_absolute_error, mean_squared_error
from sklearn.model_selection import RandomizedSearchCV
from catboost import CatBoostRegressor
from xgboost import XGBRegressor
import warnings

**Import the CSV Data as Pandas DataFrame**

In [14]:
df = pd.read_csv('stud.csv')

**Top 5 records**

In [15]:
df.head()

,gender,race_ethnicity,parental_level_of_education,lunch,test_preparation_course,math_score,reading_score,writing_score
0,female,group B,bachelor's degree,standard,none,72,72,74
1,female,group C,some college,standard,completed,69,90,88
2,female,group B,master's degree,standard,none,90,95,93
3,male,group A,associate's degree,free/reduced,none,47,57,44
4,male,group C,some college,standard,none,76,78,75


**Preparing X and y variables**

In [16]:
X = df.drop(columns=['math_score'],axis=1)

In [17]:
X.head()

,gender,race_ethnicity,parental_level_of_education,lunch,test_preparation_course,reading_score,writing_score
0,female,group B,bachelor's degree,standard,none,72,74
1,female,group C,some college,standard,completed,90,88
2,female,group B,master's degree,standard,none,95,93
3,male,group A,associate's degree,free/reduced,none,57,44
4,male,group C,some college,standard,none,78,75


In [23]:
print('Categories in gender: ', end=" ")
print(df["gender"].unique())

print("Categories in race/enthnicity: ",end=" ")
print(df["race_ethnicity"].unique())

print('Categories in lunch: ',end=" ")
print(df["lunch"].unique())

print("Categories in parents education level: ",end=" ")
print(df["parental_level_of_education"].unique())

print("Categories in test prepartion course: ", end=" ")
print( df["test_preparation_course"].unique())

Categories in gender:  ['female' 'male']
Categories in race/enthnicity:  ['group B' 'group C' 'group A' 'group D' 'group E']
Categories in lunch:  ['standard' 'free/reduced']
Categories in parents education level:  ["bachelor's degree" 'some college' "master's degree" "associate's degree"
 'high school' 'some high school']
Categories in test prepartion course:  ['none' 'completed']


In [24]:
y = df["math_score"]

In [25]:
y

0      72
1      69
2      90
3      47
4      76
       ..
995    88
996    62
997    59
998    68
999    77
Name: math_score, Length: 1000, dtype: int64

In [27]:
num_features = X.select_dtypes(exclude="object").columns
cat_features = X.select_dtypes(include="object").columns

from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.compose import ColumnTransformer

numeric_transformer = StandardScaler()
oh_transformer = OneHotEncoder()

preprocessor = ColumnTransformer(
    [
     ("OneHotEncoder",oh_transformer,cat_features),
     ("StandardScaler",numeric_transformer,num_features)
    ]
)

In [28]:
X = preprocessor.fit_transform(X)

In [29]:
X.shape

(1000, 19)

In [30]:
from sklearn.model_selection import train_test_split
X_train,X_test,y_train,y_test = train_test_split(X,y,test_size=0.2,random_state=42)
X_train.shape,X_test.shape

((800, 19), (200, 19))

**Create an Evaluate Function to give all metrics after model Training**

In [64]:
def evaluvate_model(true,predicted):
    mae = mean_absolute_error(true,predicted)
    mse = mean_squared_error(true,predicted)
    rmse = np.sqrt(mean_squared_error(true, predicted))
    r2_squre = r2_score(true,predicted)
    return mae,mse,r2_squre

In [88]:
models = {
    "Linear Regression": LinearRegression(),
    "Lasso": Lasso(),
    "Ridge": Ridge(),
    "K Neighbors Regressor": KNeighborsRegressor(),
    "Decision Tree Regressor": DecisionTreeRegressor(),
    "Random Forest Regressor": RandomForestRegressor(),
    "XGB Regressor": XGBRegressor(),
    "CatBoost Regressor": CatBoostRegressor(verbose=False),
    "AdaBoost Regressor": AdaBoostRegressor()
}

model_list = []
r2_list = []


In [90]:
list(models.values())

[LinearRegression(),
 Lasso(),
 Ridge(),
 KNeighborsRegressor(),
 DecisionTreeRegressor(),
 RandomForestRegressor(),
 XGBRegressor(base_score=None, booster=None, colsample_bylevel=None,
              colsample_bynode=None, colsample_bytree=None, gamma=None,
              gpu_id=None, importance_type='gain', interaction_constraints=None,
              learning_rate=None, max_delta_step=None, max_depth=None,
              min_child_weight=None, missing=nan, monotone_constraints=None,
              n_estimators=100, n_jobs=None, num_parallel_tree=None,
              random_state=None, reg_alpha=None, reg_lambda=None,
              scale_pos_weight=None, subsample=None, tree_method=None,
              validate_parameters=None, verbosity=None),
 AdaBoostRegressor()]

In [93]:

for i in range(len(list(models))):
    model = list(models.values())[i]
    model.fit(X_train,y_train) #Train model
    #Make predictions
    y_train_pred = model.predict(X_train)
    y_test_pred = model.predict(X_test)
    
    #Evaluvate train and test dataset
    model_train_mae, model_train_mse, model_train_r2 = evaluvate_model(y_train,y_train_pred)
    model_test_mae, model_test_mse, model_test_r2 = evaluvate_model(y_test,y_test_pred)
    model_train_rmse = np.sqrt(model_train_mse)
    model_test_rmse = np.sqrt(model_train_mse)
    
    print(list(models.keys())[i])
    model_list.append(list(models.keys())[i])
    
    print('Model performance for training set')
    print('- Root mean squared error: {:.4f}'.format(model_train_rmse))
    print('- Mean absolute error: {:.4f}'.format(model_train_mae))
    print('- R2 score : {:.4f}'.format(model_train_r2))
    
    print('****************************************************')
    
    print("Model performance for test data")
    print('- Root mean squared error: {:.4f}'.format(model_test_rmse))
    print('- Mean absolute error: {:.4f}'.format(model_train_mae))
    print('- R2 score: {:.4f}'.format(model_test_r2))
    r2_list.append(model_test_r2)
    
    print('='*35)
    print('\n')
    

Linear Regression
Model performance for training set
- Root mean squared error: 5.3303
- Mean absolute error: 4.2732
- R2 score : 0.8740
****************************************************
Model performance for test data
- Root mean squared error: 5.3303
- Mean absolute error: 4.2732
- R2 score: 0.8781


Lasso
Model performance for training set
- Root mean squared error: 6.5938
- Mean absolute error: 5.2063
- R2 score : 0.8071
****************************************************
Model performance for test data
- Root mean squared error: 6.5938
- Mean absolute error: 5.2063
- R2 score: 0.8253


Ridge
Model performance for training set
- Root mean squared error: 5.3233
- Mean absolute error: 4.2650
- R2 score : 0.8743
****************************************************
Model performance for test data
- Root mean squared error: 5.3233
- Mean absolute error: 4.2650
- R2 score: 0.8806


K Neighbors Regressor
Model performance for training set
- Root mean squared error: 5.7123
- Mean absol

In [94]:
pd.DataFrame(list(zip(model_list,r2_list)), columns=['Model_Name', 'R2_score']).sort_values(by=["R2_score"], ascending= False)

,Model_Name,R2_score
2,Ridge,0.880593
0,Linear Regression,0.878104
5,Random Forest Regressor,0.853779
7,CatBoost Regressor,0.851632
8,AdaBoost Regressor,0.847867
1,Lasso,0.825320
6,XGB Regressor,0.821589
3,K Neighbors Regressor,0.780645
4,Decision Tree Regressor,0.749752
